In [1]:
import pandas as pd
import numpy as np
from math import erf
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from datetime import datetime as dt
import os,sys
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
facts_df = pd.read_csv("../facts.csv",parse_dates=['date'],date_parser=lambda x: dt.strptime(x, '%Y-%m-%d'),index_col='date')

C:\Users\pcah8798\AppData\Local\Temp\ipykernel_21424\1148403063.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  facts_df = pd.read_csv("../facts.csv",parse_dates=['date'],date_parser=lambda x: dt.strptime(x, '%Y-%m-%d'),index_col='date')


In [3]:
facts_df.columns

Index(['harris_mean_ev', 'harris_mean_ev_lower', 'harris_mean_ev_upper',
       'trump_mean_ev', 'trump_mean_ev_lower', 'trump_mean_ev_upper',
       'harris_ev_prob', 'trump_ev_prob', 'ev_tie_prob',
       'harris_mean_popular_vote', 'harris_mean_popular_vote_lower',
       'harris_mean_popular_vote_upper', 'trump_mean_popular_vote',
       'trump_mean_popular_vote_lower', 'trump_mean_popular_vote_upper',
       'other_mean_popular_vote', 'other_mean_popular_vote_lower',
       'other_mean_popular_vote_upper', 'harris_popular_prob',
       'trump_popular_prob', 'other_popular_prob', 'electoral_college_gap',
       'electoral_college_gap_lower', 'electoral_college_gap_upper',
       'ev_popular_split_for_trump_prob', 'ev_popular_split_for_harris_prob'],
      dtype='object')

In [4]:
republican_red = "#cf1313"
democratic_blue = "#116dc2"
other_grey = "#8a8a8a"
republican_red_alphaed = "rgba(207, 19, 19, 0.2)"
democratic_blue_alphaed = "rgba(17, 109, 194, 0.2)"
other_grey_alphaed = "rgba(138, 138, 138,0.2)"

color_mapper = {"Kamala Harris":democratic_blue,"Harris":democratic_blue,"Donald Trump":republican_red,"Trump":republican_red,"Others":other_grey,"Other":other_grey}
color_mapper_alphaed = {"Kamala Harris":democratic_blue_alphaed,"Harris":democratic_blue_alphaed,"Donald Trump":republican_red_alphaed,"Trump":republican_red_alphaed,"Others":other_grey_alphaed,"Other":other_grey_alphaed}

In [5]:
the_lines = []
the_cis = []
the_max = 0
for candidate in ["Harris","Trump"]:
    date_range = facts_df[f"{candidate.lower()}_mean_ev"].index
    text = facts_df[f"{candidate.lower()}_mean_ev"].apply(lambda x: f"{candidate}: {np.round(x,1)}").values

    line = go.Scatter(
        x=date_range,
        y=facts_df[f"{candidate.lower()}_mean_ev"].values,
        mode='lines',
        line=dict(color=color_mapper[candidate], width=2),
        name='Line Data',
        hoverinfo='text',
        text=text,
        hovertemplate='%{text}<extra></extra>',
    )

    confidence_interval = go.Scatter(
        x=np.concatenate([date_range, date_range[::-1]]),
        y=np.concatenate([facts_df[f"{candidate.lower()}_mean_ev_upper"].values, (facts_df[f"{candidate.lower()}_mean_ev_lower"].values)[::-1]]),
        fill='toself',
        fillcolor=color_mapper_alphaed[candidate],
        line=dict(color='rgba(255, 255, 255, 0)'),  # No outline
        hoverinfo="skip",
        showlegend=False,
        name='Confidence Interval'
    )

    the_lines.append(line)
    the_cis.append(confidence_interval)

fig = go.Figure(data=(the_cis + the_lines))#, confidence_interval, line])

# # Customize layout
fig.update_layout(
    template="plotly_white",
    showlegend=False,
    hovermode="x",  
    dragmode=False,
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    margin={"r":0,"t":0,"l":0,"b":0},  # Adjust margins to remove extra space
)
fig.update_xaxes(range=[facts_df.index.min(), dt.strptime("2024-11-05","%Y-%m-%d")])
fig.update_yaxes(
    range=[0, 538],
    tickvals=[0,100,200,269,338,438,538], 
)
# fig.show()
fig.write_html(f"../docs/assets/ev_plot.html",config={'displayModeBar': False,"responsive": True,"scrollZoom":False})

In [6]:
the_lines = []
the_max = 0
for candidate in ["Harris","Trump"]:
    date_range = facts_df[f"{candidate.lower()}_mean_ev"].index
    text = facts_df[f"{candidate.lower()}_ev_prob"].apply(lambda x: f"{candidate}: {np.round(x*100,1)}%").values

    line = go.Scatter(
        x=date_range,
        y=facts_df[f"{candidate.lower()}_ev_prob"].values,
        mode='lines',
        line=dict(color=color_mapper[candidate], width=2),
        name='Line Data',
        hoverinfo='text',
        text=text,
        hovertemplate='%{text}<extra></extra>',
    )

    
    the_lines.append(line)


date_range = facts_df["ev_tie_prob"].index
text = facts_df["ev_tie_prob"].apply(lambda x: f"Electoral College Tie: {np.round(x*100,1)}%").values

line = go.Scatter(
    x=date_range,
    y=facts_df["ev_tie_prob"].values,
    mode='lines',
    line=dict(color=color_mapper["Others"], width=2),
    name='Line Data',
    hoverinfo='text',
    text=text,
    hovertemplate='%{text}<extra></extra>',
)


the_lines.append(line)



fig = go.Figure(data=(the_lines))#, confidence_interval, line])

# # Customize layout
fig.update_layout(
    template="plotly_white",
    showlegend=False,
    hovermode="x",  
    dragmode=False,
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    margin={"r":0,"t":0,"l":0,"b":0},  # Adjust margins to remove extra space
)
fig.update_xaxes(range=[facts_df.index.min(), dt.strptime("2024-11-05","%Y-%m-%d")])
fig.update_yaxes(
    range=[0, 1],
    tickvals=[0,0.2,0.4,0.6,0.8,1],
    ticktext=["0%","20%","40%","60%","80%","100%"]

)
# fig.show()
fig.write_html(f"../docs/assets/win_probs.html",config={'displayModeBar': False,"responsive": True,"scrollZoom":False})

In [7]:
the_lines = []
the_cis = []
the_max = 0
for candidate in ["Harris","Trump","Other"]:
    date_range = facts_df[f"{candidate.lower()}_mean_popular_vote"].index
    text = facts_df[f"{candidate.lower()}_mean_popular_vote"].apply(lambda x: f"{candidate}: {np.round(x*100,1)}%").values

    line = go.Scatter(
        x=date_range,
        y=facts_df[f"{candidate.lower()}_mean_popular_vote"].values,
        mode='lines',
        line=dict(color=color_mapper[candidate], width=2),
        name='Line Data',
        hoverinfo='text',
        text=text,
        hovertemplate='%{text}<extra></extra>',
    )

    confidence_interval = go.Scatter(
        x=np.concatenate([date_range, date_range[::-1]]),
        y=np.concatenate([facts_df[f"{candidate.lower()}_mean_popular_vote_upper"].values, (facts_df[f"{candidate.lower()}_mean_popular_vote_lower"].values)[::-1]]),
        fill='toself',
        fillcolor=color_mapper_alphaed[candidate],
        line=dict(color='rgba(255, 255, 255, 0)'),  # No outline
        hoverinfo="skip",
        showlegend=False,
        name='Confidence Interval'
    )

    the_lines.append(line)
    the_cis.append(confidence_interval)

fig = go.Figure(data=(the_cis + the_lines))#, confidence_interval, line])

# # Customize layout
fig.update_layout(
    template="plotly_white",
    showlegend=False,
    hovermode="x",  
    dragmode=False,
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    margin={"r":0,"t":0,"l":0,"b":0},  # Adjust margins to remove extra space
)
fig.update_xaxes(range=[facts_df.index.min(), dt.strptime("2024-11-05","%Y-%m-%d")])
fig.update_yaxes(
    range=[0, 0.6],
    tickvals=[0,0.1,0.2,0.3,0.4,0.5,0.6],
    ticktext=["0%","10%","20%","30%","40%","50%","60%"]

)
# fig.show()
fig.write_html(f"../docs/assets/popular_votes.html",config={'displayModeBar': False,"responsive": True,"scrollZoom":False})

In [8]:
the_lines = []
the_max = 0
for candidate in ["Harris","Trump","Other"]:
    date_range = facts_df[f"{candidate.lower()}_popular_prob"].index
    text = facts_df[f"{candidate.lower()}_popular_prob"].apply(lambda x: f"{candidate}: {np.round(x*100,1)}%").values

    line = go.Scatter(
        x=date_range,
        y=facts_df[f"{candidate.lower()}_popular_prob"].values,
        mode='lines',
        line=dict(color=color_mapper[candidate], width=2),
        name='Line Data',
        hoverinfo='text',
        text=text,
        hovertemplate='%{text}<extra></extra>',
    )

    
    the_lines.append(line)


# date_range = facts_df["harris_popular_prob"].index
# text = facts_df["harris_popular_prob"].apply(lambda x: f"Electoral College Tie: {np.round(x*100,1)}%").values

# line = go.Scatter(
#     x=date_range,
#     y=facts_df["harris_popular_prob"].values,
#     mode='lines',
#     line=dict(color=color_mapper["Others"], width=2),
#     name='Line Data',
#     hoverinfo='text',
#     text=text,
#     hovertemplate='%{text}<extra></extra>',
# )


# the_lines.append(line)



fig = go.Figure(data=(the_lines))#, confidence_interval, line])

# # Customize layout
fig.update_layout(
    template="plotly_white",
    showlegend=False,
    hovermode="x",  
    dragmode=False,
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    margin={"r":0,"t":0,"l":0,"b":0},  # Adjust margins to remove extra space
)
fig.update_xaxes(range=[facts_df.index.min(), dt.strptime("2024-11-05","%Y-%m-%d")])
fig.update_yaxes(
    range=[0, 1],
    tickvals=[0,0.2,0.4,0.6,0.8,1],
    ticktext=["0%","20%","40%","60%","80%","100%"]

)
# fig.show()
fig.write_html(f"../docs/assets/popular_probs.html",config={'displayModeBar': False,"responsive": True,"scrollZoom":False})

In [9]:

the_max = 0
date_range = facts_df[f"electoral_college_gap"].index

def ec_gap(x):
    if x>0:
        return f"Harris +{np.round(x*100,1)}%"
    if x<0:
        return f"Trump +{-np.round(x*100,1)}%"



text = facts_df[f"electoral_college_gap"].apply(ec_gap).values

line = go.Scatter(
    x=date_range,
    y=facts_df[f"electoral_college_gap"].values,
    mode='lines',
    line=dict(color=color_mapper["Trump"], width=2),
    name='Line Data',
    hoverinfo='text',
    text=text,
    hovertemplate='%{text}<extra></extra>',
)

confidence_interval = go.Scatter(
    x=np.concatenate([date_range, date_range[::-1]]),
    y=np.concatenate([facts_df[f"electoral_college_gap_upper"].values, (facts_df[f"electoral_college_gap_lower"].values)[::-1]]),
    fill='toself',
    fillcolor=color_mapper_alphaed[candidate],
    line=dict(color='rgba(255, 255, 255, 0)'),  # No outline
    hoverinfo="skip",
    showlegend=False,
    name='Confidence Interval'
)


fig = go.Figure(data=(confidence_interval,line))#, confidence_interval, line])

# # Customize layout
fig.update_layout(
    template="plotly_white",
    showlegend=False,
    hovermode="x",  
    dragmode=False,
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    margin={"r":0,"t":0,"l":0,"b":0},  # Adjust margins to remove extra space
)
fig.update_xaxes(range=[facts_df.index.min(), dt.strptime("2024-11-05","%Y-%m-%d")])
fig.update_yaxes(
    range=[-0.05, 0.05],
    tickvals=[-0.04,-0.02,0,0.02,0.04],
    ticktext=["R +4","R +2","Even","D +2","D +4"] 
)
# fig.show()
fig.write_html(f"../docs/assets/ec_gap.html",config={'displayModeBar': False,"responsive": True,"scrollZoom":False})

In [10]:
the_lines = []
the_max = 0
for candidate in ["Harris","Trump"]:
    date_range = facts_df[f"ev_popular_split_for_{candidate.lower()}_prob"].index
    text = facts_df[f"ev_popular_split_for_{candidate.lower()}_prob"].apply(lambda x: f"{candidate}: {np.round(x*100,1)}%").values

    line = go.Scatter(
        x=date_range,
        y=facts_df[f"ev_popular_split_for_{candidate.lower()}_prob"].values,
        mode='lines',
        line=dict(color=color_mapper[candidate], width=2),
        name='Line Data',
        hoverinfo='text',
        text=text,
        hovertemplate='%{text}<extra></extra>',
    )

    
    the_lines.append(line)


# 'ev_popular_split_for_trump_prob', 'ev_popular_split_for_harris_prob'



fig = go.Figure(data=(the_lines))#, confidence_interval, line])

# # Customize layout
fig.update_layout(
    template="plotly_white",
    showlegend=False,
    hovermode="x",  
    dragmode=False,
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    margin={"r":0,"t":0,"l":0,"b":0},  # Adjust margins to remove extra space
)
fig.update_xaxes(range=[facts_df.index.min(), dt.strptime("2024-11-05","%Y-%m-%d")])
fig.update_yaxes(
    range=[0, 0.4],
    tickvals=[0,0.1,0.2,0.3,0.4],
    ticktext=["0%","10%","20%","30%","40%"]

)
# fig.show()
fig.write_html(f"../docs/assets/mismatch_probs.html",config={'displayModeBar': False,"responsive": True,"scrollZoom":False})

In [11]:
facts_today = facts_df.iloc[[-1]].copy()
facts_today

,harris_mean_ev,harris_mean_ev_lower,harris_mean_ev_upper,trump_mean_ev,trump_mean_ev_lower,trump_mean_ev_upper,harris_ev_prob,trump_ev_prob,ev_tie_prob,harris_mean_popular_vote,...,other_mean_popular_vote_lower,other_mean_popular_vote_upper,harris_popular_prob,trump_popular_prob,other_popular_prob,electoral_college_gap,electoral_college_gap_lower,electoral_college_gap_upper,ev_popular_split_for_trump_prob,ev_popular_split_for_harris_prob
date,,,,,,,,,,,,,,,,,,,,,
2024-10-01,290.4206,205.98171,374.85949,247.5794,163.14051,332.01829,0.677,0.3159,0.0071,0.48615,...,0.0016,0.11567,0.855,0.145,0.0,-0.019441,-0.04586,0.006979,0.1777,0.0061


In [12]:
splash_text = ['harris_ev_prob','trump_ev_prob','ev_tie_prob'][facts_today.iloc[-1][['harris_ev_prob','trump_ev_prob','ev_tie_prob']].argmax()][:-8].capitalize()

winner_prob = max(facts_today.iloc[-1][['harris_ev_prob','trump_ev_prob','ev_tie_prob']])
if winner_prob > 0.9:
    splash_text+=" is very likely to win."
elif winner_prob > 0.8:
    splash_text+=" is very favoured to win."
elif winner_prob >0.65:
    splash_text+=" is favoured to win."
elif winner_prob >0.55:
    splash_text+=" is slightly favoured to win."
else:
    splash_text="It's a tossup."

facts_today['splash_text']=splash_text

In [13]:
facts_today.to_csv("../docs/_data/splash_text.csv")

In [14]:
facts_today['splash_text']

date
2024-10-01    Harris is favoured to win.
Name: splash_text, dtype: object